# Exemplos de tools funcionais

Agora vamos abordar a criação de duas tools com utilidade real para você entender como seria todo o processo.

## Criando um tool de busca de temperatura

https://open-meteo.com/en/docs

In [2]:
import requests
import datetime

from langchain.agents import tool
from pydantic import BaseModel, Field #Importação atualizada

class RetornTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')


@tool(args_schema=RetornTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para uma dada coordenada'''

    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))

        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return temp_atual
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')


In [3]:
retorna_temperatura_atual

StructuredTool(name='retorna_temperatura_atual', description='Retorna a temperatura atual para uma dada coordenada', args_schema=<class '__main__.RetornTempArgs'>, func=<function retorna_temperatura_atual at 0x7fd8ae98d1c0>)

In [4]:
retorna_temperatura_atual.args

{'latitude': {'description': 'Latitude da localidade que buscamos a temperatura',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude da localidade que buscamos a temperatura',
  'title': 'Longitude',
  'type': 'number'}}

### Testando a tool

In [5]:
retorna_temperatura_atual.invoke({'latitude': -30, 'longitude': -50})

28.3

## Criando um tool de busca no wikipedia

In [6]:
import wikipedia
wikipedia.set_lang('pt')

@tool
def busca_wikipedia(query: str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)

In [7]:
busca_wikipedia.invoke({'query': 'futebol'})

'Título da página: Futebol\nResumo: O futebol (em inglês: football, soccer) - originalmente futebol associado (em inglês: association football), e também referido como futebol de campoou futebol de onze - é um esporte de equipe jogado entre dois times de 11 jogadores cada um e um árbitro que se ocupa da correta aplicação das normas. É considerado o desporto mais popular do mundo, pois cerca de 270 milhões de pessoas participam das suas várias competições. É jogado num campo retangular gramado, com uma baliza em cada lado do campo. O objetivo do jogo é deslocar uma bola através do campo para colocá-la dentro da baliza adversária, ação que se denomina gol(pt-BR) ou golo(pt-PT?). A equipe que marca mais gols ao término da partida é a vencedora.\nO jogo moderno foi criado na Inglaterra com a formação de The Football Association, cujas regras de 1863 são a base do desporto na atualidade. O órgão regente do futebol é a Federação Internacional de Futebol (em francês: Fédération Internationale

### Testando a tool

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])
chat = ChatOpenAI()

tools = [convert_to_openai_function(busca_wikipedia), convert_to_openai_function(retorna_temperatura_atual)]
chain = prompt | chat.bind(functions=tools)


In [9]:
chain.invoke({'input': 'Olá'})

AIMessage(content='Olá! Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 126, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fa585e2f-82f3-453c-bd18-b5bcaad544c5-0', usage_metadata={'input_tokens': 126, 'output_tokens': 13, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
chain.invoke({'input': 'Qual é a temperatura de Porto Alegre?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-30.0346,"longitude":-51.2177}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 134, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-0e93ad1f-fc9f-4cac-bb1e-51cb0dabd08c-0', usage_metadata={'input_tokens': 134, 'output_tokens': 29, 'total_tokens': 163, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
chain.invoke({'input': 'Quem foi Isaac Asimov?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Asimov"}', 'name': 'busca_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 132, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-270fc565-b7e5-42a2-9ad4-3880589e50ca-0', usage_metadata={'input_tokens': 132, 'output_tokens': 21, 'total_tokens': 153, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})